In [1]:
# parameters_dynamic2.ipynb

import geopandas as gdp
import numpy as np
import pandas as pd
import itertools
import sys, trace
import matplotlib.pyplot as plt
from scenarios import scenarios  # Import scenario configurations
from scipy.spatial.distance import cdist
import os


In [2]:

# Select Scenario
scenario_name = "terkidi_baseline"  # Change this to switch scenarios
params = scenarios[scenario_name]  # Load selected scenario parameters


In [4]:

# Load the GeoJSON file
# location_nodes = gdp.read_file("location_nodes.geojson")
#location_nodes = gdp.read_file("location_refcamps.geojson")
location_nodes =  gdp.read_file(params["location_file"])
location_nodes


,Name,Camp,type_f,id,geometry
0,Tierkidi1,Tierkidi,demand_point,dp1,POINT (34.28023 8.28126)
1,Tierkidi2,Tierkidi,demand_point,dp2,POINT (34.2971 8.27543)
2,Tierkidi3,Tierkidi,demand_point,dp3,POINT (34.28673 8.27377)
3,Tierkidi4,Tierkidi,demand_point,dp4,POINT (34.30365 8.2793)
4,Tierkidi5,Tierkidi,demand_point,dp5,POINT (34.27802 8.27741)
...,...,...,...,...,...
112,HC_CandidateLocation3,Tierkidi,HC,hc3,POINT (34.28141 8.27974)
113,HC_CandidateLocation4,Tierkidi,HC,hc4,POINT (34.28596 8.2732)
114,HC_CandidateLocation5,Tierkidi,HC,hc5,POINT (34.29226 8.2789)
115,HC_CandidateLocation6,Tierkidi,HC,hc6,POINT (34.27802 8.27741)


In [5]:

# Add x and y coordinates
location_nodes.loc[:, 'x'] = location_nodes.geometry.x
location_nodes.loc[:, 'y'] = location_nodes.geometry.y

# Sort 
location_nodes = location_nodes.sort_values(by=['y', 'x']).reset_index(drop=True)

# Label sorted demand points
demand_points_gdf = location_nodes.loc[location_nodes.type_f == "demand_point"].copy()
demand_points_gdf['label'] = ['i' + str(i + 1) for i in range(len(demand_points_gdf))]

# Save demand point labels to a Numpy Array
dps = demand_points_gdf['label'].to_numpy()

# Subset location types
hps_gdf = location_nodes[location_nodes.type_f == "HP"]
hcs_gdf = location_nodes[location_nodes.type_f == "HC"]
hfs_gdf = location_nodes[(location_nodes.type_f == "HC") | (location_nodes.type_f == "HP")].drop_duplicates(subset='geometry').reset_index(drop=False)

# Label candidate locations
hfs_gdf['label'] = ['j' + str(j + 1) for j in range(len(hfs_gdf))]

# Save location labels
hfs = hfs_gdf['label'].to_numpy()
hps = hfs_gdf[hfs_gdf['geometry'].isin(hps_gdf['geometry'])]['label'].to_numpy()
hcs = hfs_gdf[hfs_gdf['geometry'].isin(hcs_gdf['geometry'])]['label'].to_numpy()
 

In [6]:
demand_points_gdf

,Name,Camp,type_f,id,geometry,x,y,label
0,Tierkidi35,Tierkidi,demand_point,dp35,POINT (34.28784 8.26933),34.287837,8.269329,i1
1,Tierkidi99,Tierkidi,demand_point,dp99,POINT (34.28667 8.26995),34.286670,8.269951,i2
2,Tierkidi49,Tierkidi,demand_point,dp49,POINT (34.28752 8.27056),34.287517,8.270560,i3
3,Tierkidi27,Tierkidi,demand_point,dp27,POINT (34.28633 8.27079),34.286331,8.270790,i4
4,Tierkidi77,Tierkidi,demand_point,dp77,POINT (34.2859 8.27136),34.285902,8.271361,i5
...,...,...,...,...,...,...,...,...
112,Tierkidi1,Tierkidi,demand_point,dp1,POINT (34.28023 8.28126),34.280233,8.281262,i96
113,Tierkidi72,Tierkidi,demand_point,dp72,POINT (34.27934 8.28136),34.279343,8.281360,i97
114,Tierkidi97,Tierkidi,demand_point,dp97,POINT (34.28166 8.28167),34.281662,8.281665,i98
115,Tierkidi94,Tierkidi,demand_point,dp94,POINT (34.28104 8.28235),34.281036,8.282350,i99


In [7]:
camps = set(location_nodes["Camp"].unique())
camps
camp_demand_labels = demand_points_gdf.groupby("Camp")["label"].apply(set).to_dict()
camp_demand_labels
camp_candidate_location_labels = hfs_gdf.groupby("Camp")["label"].apply(set).to_dict()
camp_candidate_location_labels 

{'Tierkidi': {'j1',
  'j10',
  'j11',
  'j12',
  'j13',
  'j14',
  'j15',
  'j2',
  'j3',
  'j4',
  'j5',
  'j6',
  'j7',
  'j8',
  'j9'}}

In [8]:
# This is _just_ to have insights on which values would be adequate for t'max and t''max 

from shapely.geometry import Point

# Initialize dictionaries to store results
avg_dist_demand_to_hp = {}
avg_dist_demand_to_hc = {}
min_intercamp_distance = {} # For distances between different camps
max_withincamp_distance = {} # For maximum distance within the same camp

# Compute distances for each camp
for camp in camps:
    # Filter locations by camp
    camp_demand_points = demand_points_gdf[demand_points_gdf["Camp"] == camp]
    camp_hps = hps_gdf[hps_gdf["Camp"] == camp]
    camp_hcs = hcs_gdf[hcs_gdf["Camp"] == camp]

    # Compute distances between demand points and HPs
    if not camp_hps.empty:
        avg_distances_per_demand_point_to_hp = camp_demand_points.to_crs(epsg=3857).geometry.apply(lambda dp: camp_hps.to_crs(epsg=3857).geometry.distance(dp).mean())
        avg_dist_demand_to_hp[camp] = avg_distances_per_demand_point_to_hp.mean()
    else:
        avg_dist_demand_to_hp[camp] = None  # No HPs in this camp

    # Compute distances between demand points and HCs
    if not camp_hcs.empty:
        avg_distances_per_demand_point_to_hc = camp_demand_points.to_crs(epsg=3857).geometry.apply(lambda dp: camp_hcs.to_crs(epsg=3857).geometry.distance(dp).mean())
        avg_dist_demand_to_hc[camp] = avg_distances_per_demand_point_to_hc.mean()
    else:
        avg_dist_demand_to_hc[camp] = None  # No HCs in this camp

    # Now calculate maximum within-camp distance (distance between any two locations within the same camp)
    locations_camp = location_nodes[location_nodes["Camp"] == camp]
    
    # Reproject to EPSG:3857 (meters) and compute the maximum pairwise distance within the camp
    max_within_distance = locations_camp.to_crs(epsg=3857).geometry.apply(
        lambda loc: locations_camp.to_crs(epsg=3857).geometry.distance(loc).max()
    ).max()
    
    # Store the maximum distance within the camp
    max_withincamp_distance[camp] = max_within_distance


# Compute minimum distance between any location in different camps
if len(camps)>1:
    for camp1 in camps:
        for camp2 in camps:
            if camp1 != camp2:
                locations_camp1 = location_nodes[location_nodes["Camp"] == camp1]
                locations_camp2 = location_nodes[location_nodes["Camp"] == camp2]
                min_distance = locations_camp1.to_crs(epsg=3857).geometry.apply(lambda loc: locations_camp2.to_crs(epsg=3857).geometry.distance(loc).min()).min()
                min_intercamp_distance[(camp1, camp2)] = min_distance

    # Now find the overall minimum distance across all inter-camp pairs
    overall_min_distance = min(min_intercamp_distance.values())

# Find the overall maximum within-camp distance
overall_max_withincamp_distance = max(max_withincamp_distance.values())


# # Convert results to DataFrames for better visualization
# df_avg_dist_hp = pd.DataFrame(list(avg_dist_demand_to_hp.items()), columns=["Camp", "Avg_Dist_Demand_to_HP"])
# df_avg_dist_hc = pd.DataFrame(list(avg_dist_demand_to_hc.items()), columns=["Camp", "Avg_Dist_Demand_to_HC"])
# df_min_intercamp = pd.DataFrame(list(overall_min_distance.items()), columns=["Camp_Pair", "Min_Distance"])
# df_max_withincamp = pd.DataFrame(list(overall_max_withincamp_distance.items()), columns=["Camp", "Max_Distance"])

# Display results
print(avg_dist_demand_to_hp)
print(avg_dist_demand_to_hc)
if len(camps)>1: print(min_intercamp_distance)
if len(camps)>1: print(overall_min_distance)
print(max_withincamp_distance)
print(overall_max_withincamp_distance)



{'Tierkidi': np.float64(1277.9341789243774)}
{'Tierkidi': np.float64(1317.5746385943735)}
{'Tierkidi': np.float64(3844.636087954079)}
3844.636087954079


In [39]:

def compute_distance_matrix(demand_points_gdf, hfs_gdf):
    """
    Compute the distance matrix between demand points and candidate health facility locations.

    Parameters:
    - demand_points_gdf: GeoDataFrame containing demand points with 'geometry'.
    - hfs_gdf: GeoDataFrame containing candidate health facility locations with 'geometry'.

    Returns:
    - distance_matrix: 2D NumPy array of distances (rows: demand points, columns: health facilities).
    """
    # Extract coordinates as NumPy arrays directly from the geometry column
    demand_coords = np.array(demand_points_gdf.geometry.apply(lambda point: (point.x, point.y)).tolist())
    hfs_coords = np.array(hfs_gdf.geometry.apply(lambda point: (point.x, point.y)).tolist())
    
    # Compute the distance matrix using cdist with Euclidean metric
    distance_matrix = cdist(demand_coords, hfs_coords, metric='euclidean')
    
    # Create a labeled DataFrame
    distance_df = pd.DataFrame(distance_matrix, index=dps, columns=hfs)

    return distance_df

In [ ]:

# Example usage:
distance_df = compute_distance_matrix(demand_points_gdf, hfs_gdf)

# To save the above matrix into an Excel file to subsequently read
# distance_df.to_excel('distance_matrix_refcamps.xlsx', sheet_name='DistanceMatrixRefCamps')#, float_format="%.2f")

# Distance matrix
# distance_matrix = pd.read_excel('distance_matrix_ij.xlsx', index_col=0)
# distance_matrix = pd.read_excel('distance_matrix_refcamps.xlsx', index_col=0)
distance_df

,j1,j2,j3,j4,j5,j6,j7,j8
i1,0.001322,0.021092,0.009975,0.013642,0.011917,0.011894,0.012079,0.014150
i2,0.000000,0.022182,0.010487,0.012342,0.010658,0.010674,0.010905,0.013015
i3,0.001043,0.021288,0.009444,0.012656,0.010840,0.010764,0.010905,0.012946
i4,0.000905,0.022456,0.010309,0.011558,0.009818,0.009807,0.010018,0.012118
i5,0.001606,0.022860,0.010408,0.010874,0.009112,0.009094,0.009305,0.011409
...,...,...,...,...,...,...,...,...
i96,0.013015,0.029968,0.015816,0.004731,0.003623,0.002889,0.002218,0.000000
i97,0.013559,0.030845,0.016690,0.004275,0.003623,0.003083,0.002654,0.000895
i98,0.012740,0.028744,0.014623,0.006036,0.004536,0.003636,0.002723,0.001485
i99,0.013620,0.029566,0.015468,0.006079,0.004897,0.004081,0.003260,0.001353


In [9]:
import numpy as np

def compute_distance_matrix_meters(demand_points_gdf, hfs_gdf, crs_epsg=3857):
    """
    Compute the distance matrix between demand points and candidate health facility locations.

    Parameters:
    - demand_points_gdf: GeoDataFrame containing the demand points with geometry (usually point geometries).
    - hfs_gdf: GeoDataFrame containing the candidate health facility locations with geometry.
    - crs_epsg: The EPSG code to which the geometries will be reprojected. Default is 3857 (Web Mercator).

    Returns:
    - distance_df: A pandas DataFrame where the rows are demand points, the columns are health facilities,
                   and the values are the distances between them.
    """
    # Reproject both demand points and health facilities to the target CRS (e.g., EPSG:3857 for meters)
    demand_points_gdf = demand_points_gdf.to_crs(epsg=crs_epsg)
    hfs_gdf = hfs_gdf.to_crs(epsg=crs_epsg)

    # Initialize an empty distance matrix with dimensions (num_demand_points x num_health_facilities)
    num_demand_points = len(demand_points_gdf)
    num_health_facilities = len(hfs_gdf)
    distance_matrix = np.zeros((num_demand_points, num_health_facilities))

    # Compute distances
    for i, demand_point in enumerate(demand_points_gdf.geometry):
        for j, hf_location in enumerate(hfs_gdf.geometry):
            distance_matrix[i, j] = demand_point.distance(hf_location)
    
    # Create a DataFrame with labeled indices and columns
    distance_df = pd.DataFrame(distance_matrix, index=dps, columns=hfs) 


    return distance_df


In [10]:

# Example usage:
distance_df = compute_distance_matrix_meters(demand_points_gdf, hfs_gdf)
distance_df

# To save the above matrix into an Excel file to subsequently read
# distance_df.to_excel('distance_matrix_refcamps_meters.xlsx', sheet_name='DistanceMatrixRefCamps')#, float_format="%.2f")
# distance_df.to_excel('distance_matrix_tierkidi.xlsx', sheet_name='DistanceMatrixTierkidi')#, float_format="%.2f")
distance_df.to_excel('distance_matrix_terkidi_baseline.xlsx', sheet_name='DistanceMatrixTierkidiBaseline')#, float_format="%.2f")

# Distance matrix
# distance_matrix = pd.read_excel('distance_matrix_ij.xlsx', index_col=0)
# distance_matrix = pd.read_excel('distance_matrix_refcamps_meters.xlsx', index_col=0)
distance_matrix = pd.read_excel('distance_matrix_terkidi_baseline.xlsx', index_col=0)
distance_matrix

,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,j11,j12,j13,j14,j15
i1,429.747320,483.009236,1608.217048,768.056994,1115.820708,983.222261,2183.725454,1357.518175,1421.591434,1618.935268,1184.000492,2087.076852,1518.195419,1372.614034,1292.294660
i2,285.970942,373.820129,1462.200515,685.722922,1171.629937,1017.917183,2276.082289,1413.890451,1277.333958,1475.681329,1183.610974,2162.857885,1380.950231,1247.391739,1291.732837
i3,326.514256,343.904184,1530.887816,625.518144,1055.129051,902.694226,2163.514305,1297.392835,1308.270437,1502.591511,1076.688136,2047.255719,1392.015269,1236.568862,1185.037446
i4,197.414829,274.043169,1396.948516,592.028918,1150.754873,983.967925,2278.915856,1391.662286,1187.738681,1384.509006,1126.128996,2152.298459,1284.210180,1146.361345,1233.444307
i5,126.121647,206.742237,1333.704432,532.821316,1161.182438,984.652418,2304.599752,1399.943846,1110.434293,1306.436839,1104.814848,2168.009149,1204.369843,1067.201595,1210.884416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i96,1188.657460,1108.508104,1021.745032,920.376303,1762.669658,1555.135850,2868.754946,1904.124300,497.830339,454.130618,1365.412274,2616.151164,252.211557,215.406128,1399.436348
i97,1244.084657,1176.770398,969.705002,1005.141737,1859.869520,1651.556811,2968.268994,2003.061643,467.508198,386.876974,1464.678402,2715.723006,218.018799,293.234931,1499.035002
i98,1172.527006,1065.746773,1162.808323,837.924003,1630.332910,1426.956933,2719.401376,1761.371977,626.693166,611.862094,1220.559453,2462.187077,397.466597,218.337010,1248.696098
i99,1267.690611,1166.170468,1173.020064,941.760753,1724.842097,1523.387475,2802.381418,1850.138855,648.688173,600.468328,1308.771456,2540.694651,403.798140,296.454611,1332.090876


In [11]:
######################################

# Health services and workers
services = ['basic','maternal1','maternal2']
health_workers = ['doctor','nurse','midwife']
levels = ['hp', 'hc']



# Assign scenario parameters
HFs_to_locate = params["HFs_to_locate"]
t1max = params["t1max"]
t2max = params["t2max"]
workers_to_allocate = params["workers_to_allocate"]
working_hours = params["working_hours"]
service_time = params["service_time"]

# Lower bound workers per HF type
lb_workers_df = pd.DataFrame(params["lb_workers"], index=health_workers)
lb_workers = {(health_workers[p], levels[l]): lb_workers_df.iloc[p, l] 
      for p, l in itertools.product(range(len(health_workers)), range(len(levels)))}

# Where can each service be provided?
services_at_HFs_df = pd.DataFrame(params["services_at_HFs"], index=services)
a_HF = {(services[s], levels[l]): services_at_HFs_df.iloc[s, l] 
      for s, l in itertools.product(range(len(services)), range(len(levels)))}

# Which health worker can deliver each service?
services_per_worker_df = pd.DataFrame(params["services_per_worker"], index=health_workers)
a_W = {(health_workers[p], services[s]): services_per_worker_df.iloc[p, s] 
      for p, s in itertools.product(range(len(health_workers)), range(len(services)))}

# Demand rates
total_population = {(key): params["total_population"] for key in dps}

# Opening hours
demand_rate_opening_hours_df = pd.DataFrame([params["demand_rate_opening_hours"]] * len(dps), index=dps, columns=services)
dr_oh = {(dps[i], services[s]): demand_rate_opening_hours_df.iloc[i, s] 
      for i, s in itertools.product(range(len(dps)), range(len(services)))}

dd_oh = {(key): int(round(total_population[i] * dr_oh[key])) for i in dps for key in dr_oh}

# Closing hours
demand_rate_closing_hours_df = pd.DataFrame([params["demand_rate_closing_hours"]] * len(dps), index=dps, columns=services)
dr_ch = {(dps[i], services[s]): demand_rate_closing_hours_df.iloc[i, s] 
      for i, s in itertools.product(range(len(dps)), range(len(services)))}

dd_ch = {(key): int(round(total_population[i] * dr_ch[key])) for i in dps for key in dr_ch}
